In [ ]:
from google.colab import files
uploaded = files.upload()
! unzip 'uw-cs480-fall20 .zip'

Streaming output truncated to the last 5000 lines.
  inflating: suffled-images/shuffled-images/5801.jpg  
  inflating: suffled-images/shuffled-images/58010.jpg  
  inflating: suffled-images/shuffled-images/58011.jpg  
  inflating: suffled-images/shuffled-images/58012.jpg  
  inflating: suffled-images/shuffled-images/58013.jpg  
  inflating: suffled-images/shuffled-images/58014.jpg  
  inflating: suffled-images/shuffled-images/58015.jpg  
  inflating: suffled-images/shuffled-images/58016.jpg  
  inflating: suffled-images/shuffled-images/58017.jpg  
  inflating: suffled-images/shuffled-images/58018.jpg  
  inflating: suffled-images/shuffled-images/5802.jpg  
  inflating: suffled-images/shuffled-images/58020.jpg  
  inflating: suffled-images/shuffled-images/58022.jpg  
  inflating: suffled-images/shuffled-images/58023.jpg  
  inflating: suffled-images/shuffled-images/58024.jpg  
  inflating: suffled-images/shuffled-images/58027.jpg  
  inflating: suffled-images/shuffled-images/58028.jpg  

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
from io import StringIO
from sklearn import preprocessing
import shutil
import os
from numpy import argmax 
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from IPython.display import Image, display
import matplotlib.pyplot as plt
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
with open('train.csv', newline='') as f:
  reader = csv.reader(f)
  data = list(reader)
ids=[row[0] for row in data[1:]]
categories=[row[1].lower() for row in data[1:]]
gender=[row[2].lower() for row in data[1:]]
base_color=[row[3].lower() for row in data[1:]]
season=[row[4].lower() for row in data[1:]]
usage=[row[5].lower() for row in data[1:]]
noisy_description=[row[6].lower() for row in data[1:]]

In [ ]:

updated_description=[]
for i in range(len(noisy_description)):
    updated_description.append(noisy_description[i].split())

dictionary={}
for i in updated_description:
    for j in i:
        if (not(j in dictionary)):
            dictionary[j]=1
        else:
            dictionary[j]+=1


for i in dictionary.copy():
    if(dictionary[i]<20 or i=="&" or i=="for" or i=="by" or len(i)<3 or i=="and" or i=="2" ):
        del dictionary[i]

dictionary['nan']=1
final_description=[]
for i in range (len(updated_description)):
    
    vocab=[]
    for j in range (len(updated_description[i])):
        if(updated_description[i][j] in dictionary):
          vocab.append(updated_description[i][j])
        else:
          vocab.append('nan')
    vocab=set(vocab)
    final_description.append(list(vocab))  


mylist=list(dictionary.keys())

text_train=np.zeros((len(final_description),len(mylist)))
print(text_train.shape)
for i in range(len(text_train)):
  for j in range(len(final_description[i])):
    text_train[i][mylist.index(final_description[i][j])]=1



(21627, 536)


In [ ]:
with open('test.csv', newline='') as f:
  reader = csv.reader(f)
  data_test = list(reader)
  ids_test=[row[0] for row in data_test[1:]]
  gender_test=[row[1].lower() for row in data_test[1:]]
  base_color_test=[row[2].lower() for row in data_test[1:]]
  season_test=[row[3].lower() for row in data_test[1:]]
  usage_test=[row[4].lower() for row in data_test[1:]]
  noisy_description_test=[row[5].lower() for row in data_test[1:]]



updated_description_test=[]
for i in range(len(noisy_description_test)):
    updated_description_test.append(noisy_description_test[i].split())

final_description_test=[]
for i in range (len(updated_description_test)):
    
    vocab=[]
    for j in range (len(updated_description_test[i])):
        if(updated_description_test[i][j] in dictionary):
          vocab.append(updated_description_test[i][j])
        else:
          vocab.append('nan')
    vocab=set(vocab)
    final_description_test.append(list(vocab))  


mylist_test=list(dictionary.keys())

text_test=np.zeros((len(final_description_test),len(mylist_test)))
print(text_test.shape)
for i in range(len(text_test)):

    for j in range(len(final_description_test[i])):
        text_test[i][mylist_test.index(final_description_test[i][j])]=1


le_categories = preprocessing.LabelEncoder()
le_categories.fit(categories)
numeric_categories=le_categories.transform(categories)

le_gender = preprocessing.LabelEncoder()
le_gender.fit(gender)
numeric_gender=le_gender.transform(gender)

le_basecolor = preprocessing.LabelEncoder()
le_basecolor.fit(base_color)
numeric_basecolor=le_basecolor.transform(base_color)

le_season = preprocessing.LabelEncoder()
le_season.fit(season)
numeric_season=le_season.transform(season)


le_usage = preprocessing.LabelEncoder()
le_usage.fit(usage)
numeric_usage=le_usage.transform(usage)


numeric_gender_test=le_gender.transform(gender_test)
numeric_basecolor_test=le_basecolor.transform(base_color_test)
numeric_season_test=le_season.transform(season_test)
numeric_usage_test=le_usage.transform(usage_test)



train_target=np.zeros((len(ids)))
for i in range(len(train_target)):
    train_target[i]=numeric_categories[i]
    
print(type(train_target))



(21628, 536)
<class 'numpy.ndarray'>


In [ ]:
import torch
train_target=torch.from_numpy(train_target)

In [ ]:


class networkt(nn.Module):
    def __init__(self):
        super(networkt, self).__init__()

        self.lin = nn.Linear(536, 1024)
        self.drop=nn.Dropout(p=0.2)
        self.lin3=nn.Linear(1024,27)
        self.softmax = nn.Softmax()
        
    def forward(self, x):

        x=F.relu(self.lin(x))
        x=self.drop(x)
        x=F.relu(self.lin3(x))
        
        
        return x

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from IPython.display import Image, display
import matplotlib.pyplot as plt
criterion = nn.CrossEntropyLoss()


# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
batch_size = 128
modelt=networkt().to(device)
optimizer = optim.Adam(modelt.parameters(), lr=1e-3)

max_epochs=500
totalloss=0
minloss=float('inf')
max_num_correct=0
num_correct=0

my_loader = torch.utils.data.DataLoader(text_train[0:21000,:],batch_size)
mytargetloader=torch.utils.data.DataLoader(train_target[0:21000],batch_size)
 
my_loader_test = torch.utils.data.DataLoader(text_train[21000:,:],batch_size)
mytargetloader_test=torch.utils.data.DataLoader(train_target[21000:],batch_size)
  
for i in range(max_epochs):
  if(num_correct>max_num_correct):
    torch.save(modelt, 'best_model_text.pt')
    max_num_correct=num_correct
  num_correct=0

  for idx,(data,target) in enumerate(zip(my_loader,mytargetloader)):
    data = data.to(device).float()   
    target=target.to(device).long()
    optimizer.zero_grad()
    predicted_batch = modelt(data)

    #pred = torch.argmax(predicted_batch,1)
    loss = criterion(predicted_batch, target)
    loss.backward()
    #totalloss+=loss.item()
    optimizer.step()
  
  for idx,(datat,targett) in enumerate(zip(my_loader_test,mytargetloader_test)):
    datat = datat.to(device).float()   
    targett=targett.to(device).long()
    predicted_batcht = modelt(datat)
    preds=torch.argmax(predicted_batcht,1)  
    num_correct+=(preds== targett).sum()
  
  print(str(i)+" "+str(num_correct))
print(max_num_correct)

0 tensor(466)
1 tensor(468)
2 tensor(468)
3 tensor(469)
4 tensor(468)
5 tensor(468)
6 tensor(470)
7 tensor(472)
8 tensor(469)
9 tensor(470)
10 tensor(469)
11 tensor(468)
12 tensor(466)
13 tensor(467)
14 tensor(470)
15 tensor(469)
16 tensor(471)
17 tensor(469)
18 tensor(466)
19 tensor(468)
20 tensor(468)
21 tensor(468)
22 tensor(470)
23 tensor(466)
24 tensor(468)
25 tensor(464)
26 tensor(468)
27 tensor(468)
28 tensor(465)
29 tensor(466)
30 tensor(466)
31 tensor(465)
32 tensor(469)
33 tensor(466)
34 tensor(467)
35 tensor(467)
36 tensor(466)
37 tensor(464)
38 tensor(463)
39 tensor(466)
40 tensor(464)
41 tensor(465)
42 tensor(466)
43 tensor(465)
44 tensor(466)
45 tensor(465)
46 tensor(466)
47 tensor(466)
48 tensor(465)
49 tensor(466)
50 tensor(467)
51 tensor(467)
52 tensor(471)
53 tensor(463)
54 tensor(462)
55 tensor(463)
56 tensor(463)
57 tensor(461)
58 tensor(468)
59 tensor(463)
60 tensor(460)
61 tensor(467)
62 tensor(470)
63 tensor(466)
64 tensor(463)
65 tensor(469)
66 tensor(465)
67 te

In [ ]:
from google.colab import files
files.download("best_model_text.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
train_dataset=np.zeros((len(ids),6))

for i in range(len(train_dataset)):
    train_dataset[i][0]=ids[i]
    train_dataset[i][1]=numeric_gender[i]
    train_dataset[i][2]=numeric_basecolor[i]
    train_dataset[i][3]=numeric_season[i]
    train_dataset[i][4]=numeric_usage[i]
    #train_dataset[i][5]=numeric_discription[i]
    train_dataset[i][5]=numeric_categories[i]



test_dataset=np.zeros((len(ids_test),5))

for i in range(len(test_dataset)):
    test_dataset[i][0]=ids_test[i]
    test_dataset[i][1]=numeric_gender_test[i]
    test_dataset[i][2]=numeric_basecolor_test[i]
    test_dataset[i][3]=numeric_season_test[i]
    test_dataset[i][4]=numeric_usage_test[i]
    #test_dataset[i][5]=numeric_discription_test[i]
    



In [ ]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from IPython.display import Image, display
import matplotlib.pyplot as plt
import os 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [ ]:
from PIL import Image
path = '/content/suffled-images/shuffled-images'
os.chdir(path)
training_image = []

for i in range(len(train_dataset)):
  imagename=str(int(train_dataset[i][0]))+'.jpg'
  pic = cv2.imread(imagename)
  training_image.append([pic])

testing_image = []
for j in range(len(test_dataset)):
  imagename=str(int(test_dataset[j][0]))+'.jpg'
  pic = cv2.imread(imagename)
  testing_image.append([pic])

np.save(os.path.join(path,'training_features'),np.array(training_image))
np.save(os.path.join(path,'test_features'),np.array(testing_image))




In [ ]:
import shutil
shutil.copy('/content/suffled-images/shuffled-images/training_features.npy','/content')
shutil.copy('/content/suffled-images/shuffled-images/test_features.npy','/content')



'/content/test_features.npy'

In [ ]:

os.chdir('/content')
arr1=np.load('training_features.npy')
arr2=np.load('test_features.npy')
arr1=arr1[:,0,:,:]
arr2=arr2[:,0,:,:]



In [ ]:

torch_arr1=torch.from_numpy(arr1).float()
torch_arr2=torch.from_numpy(arr2).float()

torch_arr1=torch_arr1.permute(0,3,1,2)
torch_arr2=torch_arr2.permute(0,3,1,2)



torch_arr1/=255
torch_arr2/=255





In [ ]:
from numpy import argmax 

class network1(nn.Module):
    def __init__(self):
        super(network1, self).__init__()

        self.lin = nn.Linear(4, 32)
        self.drop=nn.Dropout(p=0.2)
        self.lin3=nn.Linear(32,27)
        self.softmax = nn.Softmax()
        
    def forward(self, x):
        x=F.relu(self.lin(x))
        x=self.drop(x)
        x=F.relu(self.lin3(x))
        
        
        return x



      
       

In [ ]:

print(train_dataset.shape)
print(type(train_dataset.shape))
train_target=np.zeros((len(ids)))
for i in range(len(train_target)):
    train_target[i]=numeric_categories[i]
    
print(type(train_target))
train_target=torch.from_numpy(train_target)
print(train_target.shape)

(21627, 6)
<class 'tuple'>
<class 'numpy.ndarray'>
torch.Size([21627])


In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from IPython.display import Image, display
import matplotlib.pyplot as plt
criterion = nn.CrossEntropyLoss()


# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
batch_size = 128
model=network1().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

max_epochs=500
totalloss=0
minloss=float('inf')
max_num_correct=0
num_correct=0

my_loader = torch.utils.data.DataLoader(train_dataset[0:21000,1:5],batch_size)
mytargetloader=torch.utils.data.DataLoader(train_dataset[0:21000,5],batch_size)

my_loader_test = torch.utils.data.DataLoader(train_dataset[21000:,1:5],batch_size)
mytargetloader_test=torch.utils.data.DataLoader(train_dataset[21000:,5],batch_size)
  
for i in range(max_epochs):
  if(num_correct>max_num_correct):
    torch.save(model, 'best_model_cat.pt')
    max_num_correct=num_correct
  num_correct=0

  for idx,(data,target) in enumerate(zip(my_loader,mytargetloader)):
    data = data.to(device).float()   
    target=target.to(device).long()
    optimizer.zero_grad()
    predicted_batch = model(data)
    #pred = torch.argmax(predicted_batch,1)
    loss = criterion(predicted_batch, target)
    loss.backward()
    #totalloss+=loss.item()
    optimizer.step()
  
  for idx,(datat,targett) in enumerate(zip(my_loader_test,mytargetloader_test)):
    datat = datat.to(device).float()   
    targett=targett.to(device).long()
    predicted_batcht = model(datat)
    preds=torch.argmax(predicted_batcht,1) 
    num_correct+=(preds== targett).sum()
  print(str(i)+" "+str(num_correct))
print(max_num_correct)




0 tensor(193)
1 tensor(218)
2 tensor(232)
3 tensor(246)
4 tensor(247)
5 tensor(254)
6 tensor(259)
7 tensor(256)
8 tensor(269)
9 tensor(261)
10 tensor(270)
11 tensor(269)
12 tensor(267)
13 tensor(261)
14 tensor(268)
15 tensor(269)
16 tensor(263)
17 tensor(280)
18 tensor(267)
19 tensor(276)
20 tensor(274)
21 tensor(272)
22 tensor(273)
23 tensor(267)
24 tensor(271)
25 tensor(280)
26 tensor(269)
27 tensor(276)
28 tensor(274)
29 tensor(265)
30 tensor(273)
31 tensor(276)
32 tensor(283)
33 tensor(276)
34 tensor(280)
35 tensor(274)
36 tensor(291)
37 tensor(289)
38 tensor(282)
39 tensor(291)
40 tensor(282)
41 tensor(284)
42 tensor(279)
43 tensor(286)
44 tensor(275)
45 tensor(285)
46 tensor(279)
47 tensor(289)
48 tensor(283)
49 tensor(286)
50 tensor(301)
51 tensor(285)
52 tensor(279)
53 tensor(285)
54 tensor(282)
55 tensor(282)
56 tensor(283)
57 tensor(294)
58 tensor(292)
59 tensor(287)
60 tensor(277)
61 tensor(290)
62 tensor(279)
63 tensor(274)
64 tensor(285)
65 tensor(301)
66 tensor(280)
67 te

In [ ]:
from google.colab import files
files.download("best_model_cat.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
class network(nn.Module):  
    def __init__(self):
        super(network, self).__init__()     
        self.cnn_layers = Sequential(
        Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
        ReLU(inplace=True),
        MaxPool2d(kernel_size=2, stride=2),
        Dropout(0.2),
        
        Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
        ReLU(inplace=True),
        MaxPool2d(kernel_size=2, stride=2),
        Dropout(0.2),
        
        Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        ReLU(inplace=True),
        MaxPool2d(kernel_size=2, stride=2),
        Dropout(0.2),
        )
        self.lin=nn.Linear(4480,27)
          
    def forward(self, x):
        x=(self.cnn_layers(x))
        x=torch.flatten(x,1)
        x=self.lin(x)
        return x


In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from IPython.display import Image, display
import matplotlib.pyplot as plt
criterion2 = nn.CrossEntropyLoss()

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

batch_size = 128
model2=network().to(device).float()
optimizer2 = optim.Adam(model2.parameters(), lr=1e-3)

my_loader2 = torch.utils.data.DataLoader(torch_arr1[0:21000,:,:,:], batch_size)
mytargetloader2=torch.utils.data.DataLoader(train_dataset[0:21000,5], batch_size)


my_loader2_test = torch.utils.data.DataLoader(torch_arr1[21000:,:,:,:], batch_size)
mytargetloader2_test=torch.utils.data.DataLoader(train_dataset[21000:,5], batch_size)

max_num_correct2=0
num_correct2=0
max_epochs2=50
totalloss2=0
minloss2=float('inf')

for i in range(max_epochs2):
  if(num_correct2>max_num_correct2):
    torch.save(model2, 'best_model_image.pt')
    max_num_correct2=num_correct2
  num_correct2=0
  for idx,(data,target2) in enumerate(zip(my_loader2,mytargetloader2)):
    data = data.to(device).float()   
    target2=target2.to(device).long()
    
    optimizer2.zero_grad()
    predicted_batch = model2(data)

    loss2 = criterion2(predicted_batch, target2)
    loss2.backward()

    optimizer2.step()
  
  for idx,(data2t,target2t) in enumerate(zip(my_loader2_test,mytargetloader2_test)):
    data2t = data2t.to(device).float()   
    target2t=target2t.to(device).long()
    predicted_batch2t = model2(data2t)
    preds2=torch.argmax(predicted_batch2t,1)
    num_correct2+=(preds2== target2t).sum()
  
  print(str(i)+" "+ str(num_correct2))
print(max_num_correct2)

0 tensor(463)
1 tensor(500)
2 tensor(526)
3 tensor(525)
4 tensor(532)
5 tensor(544)
6 tensor(550)
7 tensor(545)
8 tensor(537)
9 tensor(549)
10 tensor(555)
11 tensor(554)
12 tensor(554)
13 tensor(559)
14 tensor(554)
15 tensor(556)
16 tensor(551)
17 tensor(555)
18 tensor(558)
19 tensor(555)
20 tensor(546)
21 tensor(555)
22 tensor(548)
23 tensor(561)
24 tensor(559)
25 tensor(559)
26 tensor(566)
27 tensor(549)
28 tensor(561)
29 tensor(555)
30 tensor(561)
31 tensor(559)
32 tensor(556)
33 tensor(551)
34 tensor(558)
35 tensor(558)
36 tensor(550)
37 tensor(558)
38 tensor(567)
39 tensor(556)
40 tensor(563)
41 tensor(558)
42 tensor(561)
43 tensor(563)
44 tensor(560)
45 tensor(559)
46 tensor(549)
47 tensor(560)
48 tensor(554)
49 tensor(552)
tensor(567)


In [ ]:
from google.colab import files
files.download("best_model_image.pt")



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
class myensemble(nn.Module):
    def __init__(self, model1, model2, model3, num_classes=27):
        super(myensemble, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model1.fc = nn.Identity()
        self.model2.fc = nn.Identity()
        self.model3.fc = nn.Identity()
        
        self.lin=nn.Linear(81,128)
        self.classifier = nn.Linear(128, num_classes)
        
        
    def forward(self, x,y,z):

        x1 = self.model1(x)  
        x1 = x1.view(x1.size(0), -1)  
        y1 = self.model2(y)
        y1 = y1.view(y1.size(0), -1)
        z1 = self.model3(z)
        z1 = z1.view(z1.size(0),-1)
        xf = torch.cat((x1, y1,z1), dim=1) 
        xf=self.lin(xf)  
        xf = self.classifier(F.relu(xf))
   
        return xf


In [ ]:
mymodel1 = torch.load('best_model_cat.pt')
mymodel3 =torch.load('best_model_text.pt')
mymodel2 =torch.load('best_model_image.pt')

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from IPython.display import Image, display
import matplotlib.pyplot as plt
criterion3 = nn.CrossEntropyLoss()
# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

batch_size = 128
model3 = myensemble(mymodel1, mymodel2, mymodel3).to(device).float()
optimizer3 = optim.Adam(model3.parameters(), lr=1e-3)

myloader31=torch.utils.data.DataLoader(train_dataset[0:21000,1:5], batch_size)
myloader32 = torch.utils.data.DataLoader(torch_arr1[0:21000,:,:,:], batch_size)
myloader33 = torch.utils.data.DataLoader(text_train[0:21000,:], batch_size)
mytargetloader3=torch.utils.data.DataLoader(train_dataset[0:21000,5], batch_size)

myloader31_test=torch.utils.data.DataLoader(train_dataset[21000:,1:5], batch_size)
myloader32_test = torch.utils.data.DataLoader(torch_arr1[21000:,:,:,:], batch_size)
myloader33_test = torch.utils.data.DataLoader(text_train[21000:,:], batch_size)
mytargetloader3_test=torch.utils.data.DataLoader(train_dataset[21000:,5], batch_size)


max_num_correct3=0
num_correct3=0

max_epochs3=50
totalloss3=20
minloss3=float('inf')
for i in range(max_epochs3):
  if(num_correct3>max_num_correct3):
    torch.save(model3, 'best_model_ensemble.pt')
    max_num_correct3=num_correct3
  num_correct3=0
  for idx,(data1,data2,data3,target3) in enumerate(zip(myloader31,myloader32,myloader33,mytargetloader3)):
    data1 = data1.to(device).float()   
    data2 = data2.to(device).float() 
    data3 = data3.to(device).float()
    target3=target3.to(device).long()
    
    optimizer3.zero_grad()
    predicted_batch = model3(data1,data2,data3)

    loss3 = criterion3(predicted_batch, target3)
    loss3.backward()
    #totalloss3+=loss3.item()
    optimizer3.step()
  
  for idx,(data1t,data2t,data3t,target3t) in enumerate(zip(myloader31_test,myloader32_test,myloader33_test,mytargetloader3_test)):
    data1t = data1t.to(device).float()   
    data2t = data2t.to(device).float() 
    data3t=data3t.to(device).float() 

    target3t=target3t.to(device).long()
    predicted_batch3t = model3(data1t,data2t,data3t)
    preds3=torch.argmax(predicted_batch3t,1)
    num_correct3+=(preds3== target3t).sum()
  
  
  print(str(i)+" "+ str(num_correct3))
print(max_num_correct3)

0 tensor(598)
1 tensor(603)
2 tensor(598)
3 tensor(601)
4 tensor(600)
5 tensor(599)
6 tensor(597)
7 tensor(602)
8 tensor(605)
9 tensor(596)
10 tensor(600)
11 tensor(604)
12 tensor(595)
13 tensor(600)
14 tensor(602)
15 tensor(600)
16 tensor(602)
17 tensor(601)
18 tensor(603)
19 tensor(600)
20 tensor(602)
21 tensor(598)
22 tensor(596)
23 tensor(599)
24 tensor(600)
25 tensor(596)
26 tensor(603)
27 tensor(600)
28 tensor(598)
29 tensor(596)
30 tensor(605)
31 tensor(603)
32 tensor(600)
33 tensor(600)
34 tensor(596)
35 tensor(608)
36 tensor(601)
37 tensor(597)
38 tensor(597)
39 tensor(601)
40 tensor(603)
41 tensor(606)
42 tensor(605)
43 tensor(604)
44 tensor(603)
45 tensor(595)
46 tensor(598)
47 tensor(600)
48 tensor(601)
49 tensor(601)
tensor(608)


In [ ]:
from google.colab import files
files.download('best_model_ensemble.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
criterion3 = nn.CrossEntropyLoss()
# run on GPU if possible
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

batch_size = 128

myfinalmodel = torch.load('best_model_ensemble.pt')
myfinalmodel.eval()


myfinalloader1=torch.utils.data.DataLoader(test_dataset[:,1:5], batch_size)
myfinalloader2 = torch.utils.data.DataLoader(torch_arr2, batch_size)
myfinalloader3 = torch.utils.data.DataLoader(text_test, batch_size)
final_prediction=np.zeros((len(test_dataset),1))
j=0
with torch.no_grad():
  
  for idx,(data1f,data2f,data3f) in enumerate(zip(myfinalloader1,myfinalloader2,myfinalloader3)):

    data1f = data1f.to(device).float()   
    data2f = data2f.to(device).float() 
    data3f = data3f.to(device).float() 

    predicted_batchf = myfinalmodel(data1f,data2f,data3f)

    preds=torch.argmax(predicted_batchf,1)
    for k in range(len(preds)):
      final_prediction[j][0]=preds[k]
      j+=1

print(final_prediction.shape)

(21628, 1)


In [ ]:
int_predicts=final_prediction.astype(int)


In [ ]:
with open('train.csv', newline='') as f:
  reader = csv.reader(f)
  data = list(reader)
categoriesf=[row[1] for row in data[1:]]
le_categoriesf = preprocessing.LabelEncoder()
le_categoriesf.fit(categoriesf)
lab=le_categoriesf.inverse_transform(int_predicts)
print(lab)

['Innerwear' 'Topwear' 'Belts' ... 'Topwear' 'Lips' 'Bags']


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
import csv
with open('20893269_Najafian8.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['id', 'category'])
    for i in range(len(test_dataset)):
      writer.writerow([ids_test[i],lab[i]])

In [ ]:
from google.colab import files
files.download("20893269_Najafian8.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>